In [33]:
import random
import numpy as np
import pandas as pd
import re

In [34]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

In [35]:
def export_dataset():
    #Lista de links (arquivos estão dentro do Google Sheets)
    test_links = {
        'link_01': 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQvClyfCtk8JD82zXYSipnN7iCJ0XG0V-z6tcII_14W7tQYPpLLOK45kKWLQf8TBfyRNhH9mlEXiowR/pub?output=csv',
        'link_02': 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQq0fLn7Dkg0-Lg2i9FUD0iL-xLVHn1Gtorx8wLlA7flMftChIJf_PXgziaHshCnxlX52bZJZ2EKFsu/pub?output=csv'
    }
    train_links ={
        'link_01':'https://docs.google.com/spreadsheets/d/e/2PACX-1vS2E4FyELPrgf1rPIeyh4lzQTOtftYYK6vyr7m5TN7cYDZTo4tLy1jZQ06mnnwo0NrJ5ZR5IOm2-ndN/pub?output=csv',
        'link_02':'https://docs.google.com/spreadsheets/d/e/2PACX-1vT-NHTDb71tp400NvyVObenotd8uUATgPgl4nlGwvqpwTTMoCmc8bH7YmyvmzBg_TcIAxoNkC0upfoe/pub?output=csv',       
        'link_03':'https://docs.google.com/spreadsheets/d/e/2PACX-1vT_vX9ELHWxTt5kPrcZ8LchIXnBrd2QvbsuA5f_cHLWhaSvoa9DERR5W7oKE9489UzfFruwyek2XJvW/pub?output=csv',
        'link_04':'https://docs.google.com/spreadsheets/d/e/2PACX-1vSi7JewvqVkONWiqbFWqyQjTOmtxrsEtZ4ArifmlCMELo9BAPw00BpnZ1BI9wx_ruE4nPbY3egzjEfx/pub?output=csv',
        'link_05':'https://docs.google.com/spreadsheets/d/e/2PACX-1vTooCcyaoXnkazJ6HI95oc-Ll45cMY4qnjh6iIa9HVJfhTej2z9EBDVVHgplEhqLfegmeucud1mXMhS/pub?output=csv'
    }
   
    #Download e leitura do arquivo de treinamento
    temp_df_test = pd.DataFrame()
    for i in test_links:
        file_temp = pd.read_csv(test_links[i])
        temp_df_test = pd.concat([temp_df_test,file_temp]) 
    temp_df_test.reset_index(drop=True, inplace=True)
    temp_df_test.drop('Unnamed: 0',axis=1, inplace= True)

    #Download e leitura do arquivo de teste
    temp_df_train = pd.DataFrame()
    for i in train_links:
        file_temp = pd.read_csv(train_links[i])
        temp_df_train = pd.concat([temp_df_train,file_temp]) 
    temp_df_train.reset_index(drop=True, inplace=True)
    temp_df_train.drop('Unnamed: 0',axis=1, inplace= True)   
   
    return temp_df_test, temp_df_train

test, train = export_dataset()

| Nome da coluna | Descrição |
| :----: | :----: |
| test_id | Numero identificador dos itens da lista |
| name | Titulo da lista*|
| item_condition_id | Numero da condição do item, varia de 1 a 5, sendo 1 a pior e 5 a melhor |
| category_name | Categorias da lista |
| brand_name | Nome da marca do item (existe nulos) |
| price | Preço do produto |
| shipping | Taxa de envio, pago por: 1 - pelo vendedor 0 - pelo comprador |
| item_description | Descrição completa do item* |
| date | Data |
| stock | Estoque |

*¹- Nessas colunas existe um valor chamado [rm], esse valor significa "removed" ou removido, significa que o preço (exemplo: $20) foi removido.

In [36]:
train_vis = train[0:1000]
test_vis = test[0:1000]

In [37]:
train.shape

(1482535, 10)

In [38]:
train.info()    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482535 entries, 0 to 1482534
Data columns (total 10 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   train_id           1482535 non-null  int64  
 1   name               1482535 non-null  object 
 2   item_condition_id  1482535 non-null  int64  
 3   category_name      1476256 non-null  object 
 4   brand_name         850368 non-null   object 
 5   price              1482535 non-null  float64
 6   shipping           1482535 non-null  int64  
 7   item_description   1482531 non-null  object 
 8   date               1482535 non-null  object 
 9   stock              1482535 non-null  int64  
dtypes: float64(1), int64(4), object(5)
memory usage: 113.1+ MB


##Limpeza de dados

In [39]:
import nltk
nltk.download('stopwords')

stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
def clear_data(df):
  # droping null categories names and brands names(around 3000 values)
  dropnull = df[df.category_name.isnull() & df.brand_name.isnull()].index
  df = df.drop(dropnull, axis=0).reset_index(drop=True)
  
  # replace null values to "Other" df['category_name'] = df['category_name'].fillna("Other")
  df['category_name'] = df['category_name'].fillna("Other")
  
  #have just 4 lines of null item description, i will remove them
  df = df.dropna(subset = ['item_description']).reset_index(drop=True)
  df = df.drop(columns = ['train_id'])
  
  #I will raplace all 629225 null values in "train['brand_name']" for other again, after we can looking again
  df['brand_name'] = df['brand_name'].fillna("Other")
  
  #replacing data 29-02-2018 for 28-02-2018
  df['date'] =  df['date'].replace('29-2-2018','28-2-2018')
  
  #Creating new dates columns
  df['date2'] = pd.to_datetime(df['date'], errors='coerce')    #corrige os erros com os valores das datas de nascimento, adequa para DateTime
  df['day'] = df['date2'].dt.day
  df['month'] = df['date2'].dt.month
  df['year'] = df['date2'].dt.year

  #It drops rows with 0 price.
  x = df.query(f"{'price'} == 0").index
  df.drop(x, inplace= True)

  return df

train = clear_data(train)

In [41]:
description = train['item_description']
name_2 = train['name']

In [42]:
#downloads necessários para rodar o código abaixo
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [43]:
# funcao para tratar cada palavra e remover stopwords
def processamento(tokens):
    
    token_processado = []
    for token in tokens:
        token = token.lower()
        token = lemmatizer.lemmatize(token)
        
        if token not in stop_words:
            token = stemmer.stem(token)
            token_processado.append(token)
        
    return token_processado

# expressao regular para remover pontuacoes do texto da coluna 
documentos_descript = []
for descript in description:
    
    # expressao regular para remover pontuacoes do texto item_description
    descript = re.sub(r'[^\w\s]','', descript)
    tokens = processamento(word_tokenize(descript))
    
    documentos_descript.append(' '.join(tokens))

In [44]:
# expressao regular para remover pontuacoes do texto da coluna name

documentos_name = []
for name_3 in name_2:
    
    
    sinopse = re.sub(r'[^\w\s]','', name_3)
    tokens = processamento(word_tokenize(name_3))
    
    documentos_name.append(' '.join(tokens))

In [45]:
#criando a coluna documento_descript
train['description_clear'] = documentos_descript
train['name_clear'] = documentos_name

In [46]:
 #remoção dos itens duplicados
index_duplicated = train[train[['category_name', 'name_clear','description_clear','date','price']].duplicated()].index
train = train.drop(index_duplicated, axis=0).reset_index(drop=True)

#Continuação